In [76]:
from datetime import datetime, timedelta
import pytz

import requests
from copy import deepcopy

import polars as pl
import numpy as np

from dotenv import load_dotenv
import os

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [77]:
load_dotenv()

True

In [78]:
apikey = os.getenv("distance_matrix_apikey")

In [79]:
arrival_location = "Edingensesteenweg 196, 1500 Halle"
departure_location = "Mechelsevest 36, 3000 Leuven"

units = "metric"

In [80]:
def extract_duration_in_traffic(response: dict) -> int:
    return response["rows"][0].get("elements")[0].get("duration_in_traffic")["value"]

In [81]:
def get_forecast_times_from_departures(
    minimum_departure_time: datetime,
    maximum_arrival_time: datetime,
    arrival_location: str,
    departure_location: str,
) -> pl.DataFrame:
    params = {
        "destinations": arrival_location,
        "origins": departure_location,
        "units": "metric",
        "key": apikey,
        "departure_time": 0,
    }

    step_size_seconds = 10 * 60

    minimum_departure_timestamp = int(minimum_departure_time.timestamp())
    current_arrival_timestamp = 0
    current_departure_timestamp = deepcopy(minimum_departure_timestamp)

    maximum_arrival_timestamp = int(maximum_arrival_time.timestamp())

    data = []

    while current_arrival_timestamp <= maximum_arrival_timestamp:
        params.update({"departure_time": current_departure_timestamp})

        response = requests.get(url, params).json()

        duration_seconds = extract_duration_in_traffic(response)

        duration_minutes = duration_seconds / 60 

        current_arrival_timestamp = current_departure_timestamp + duration_seconds

        current_arrival_time = datetime.fromtimestamp(current_arrival_timestamp)

        current_departure_time = datetime.fromtimestamp(current_departure_timestamp)

        data.append(
            {
                "departure_time": current_departure_time,
                "duration_minutes": duration_minutes,
                "arrival_time": current_arrival_time,
            }
        )

        current_departure_timestamp += step_size_seconds

    return pl.from_dicts(data)

In [82]:
pl_morning = get_forecast_times_from_departures(
    datetime(2024, 7, 23, 6, 0),
    datetime(2024, 7, 23, 9, 0),
    arrival_location,
    departure_location,
)

pl_morning.head()

departure_time,duration_minutes,arrival_time
datetime[μs],f64,datetime[μs]
2024-07-23 06:00:00,43.4,2024-07-23 06:43:24
2024-07-23 06:10:00,45.083333,2024-07-23 06:55:05
2024-07-23 06:20:00,49.4,2024-07-23 07:09:24
2024-07-23 06:30:00,52.916667,2024-07-23 07:22:55
2024-07-23 06:40:00,56.85,2024-07-23 07:36:51


In [83]:
pl_evening = get_forecast_times_from_departures(
    datetime(2024, 7, 23, 16, 0),
    datetime(2024, 7, 23, 20, 0),
    departure_location,
    arrival_location,
)

pl_evening.head()


departure_time,duration_minutes,arrival_time
datetime[μs],f64,datetime[μs]
2024-07-23 16:00:00,61.716667,2024-07-23 17:01:43
2024-07-23 16:10:00,63.3,2024-07-23 17:13:18
2024-07-23 16:20:00,62.483333,2024-07-23 17:22:29
2024-07-23 16:30:00,63.116667,2024-07-23 17:33:07
2024-07-23 16:40:00,62.866667,2024-07-23 17:42:52


In [84]:
pl_morning = pl_morning.with_columns(pl.lit("morning").alias("time_of_day"))
pl_evening = pl_evening.with_columns(pl.lit("evening").alias("time_of_day"))

pl_trips = pl.concat([pl_morning, pl_evening])

In [85]:
fig = make_subplots(rows=2, cols=1, subplot_titles=("Morning", "Evening"))

# Add traces for each subplot
fig.add_trace(go.Scatter(x=pl_morning["departure_time"], y=pl_morning["duration_minutes"], mode='lines', name="morning"), row=1, col=1)
fig.add_trace(go.Scatter(x=pl_evening["departure_time"], y=pl_evening["duration_minutes"], mode='lines', name="evening"), row=2, col=1)